In [6]:
# Step 4: Evaluation and Dashboard Creation
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
from imblearn.over_sampling import SMOTE
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
# Load the engineered dataset
df = pd.read_csv('mobile_money_features.csv')
print("Dataset loaded. Shape:", df.shape)

Dataset loaded. Shape: (1000, 18)


In [8]:
# Define features and target
features = [
    'Amount', 'Rolling_Count_1h', 'Time_Delta', 'Rapid_Transaction',
    'Avg_Amount', 'Max_Amount', 'Min_Amount', 'Location_Change',
    'Unique_Devices', 'Send_Money_Ratio', 'Hour_of_Day'
]
X = df[features]
y = df['Fraud_Label']

In [9]:
# Debug: Check full dataset class distribution
print("Full dataset class distribution:", np.bincount(y))

Full dataset class distribution: [1000]


In [10]:
# Split into train and test sets (70% train, 30% test for more fraud cases)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [11]:
# Debug: Check split distributions
print("Class distribution in y_train (before SMOTE):", np.bincount(y_train))
print("Class distribution in y_test:", np.bincount(y_test))

Class distribution in y_train (before SMOTE): [700]
Class distribution in y_test: [300]


In [12]:
# Apply SMOTE only if both classes are present in y_train
if len(np.unique(y_train)) > 1:
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    print("Class distribution in y_train (after SMOTE):", np.bincount(y_train_smote))
else:
    print("Warning: Only one class in y_train. Skipping SMOTE.")
    X_train_smote, y_train_smote = X_train, y_train

In [13]:
# Train XGBoost model
model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
model.fit(X_train_smote, y_train_smote)

C:\Users\G_BOOTS\anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:08:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [14]:
# Predict on test set
y_pred = model.predict(X_test)

In [15]:
# Save predictions for dashboard
df_test = X_test.copy()
df_test['Actual_Label'] = y_test
df_test['Predicted_Label'] = y_pred
df_test.to_csv('fraud_predictions.csv', index=False)

In [16]:
# --- Streamlit Dashboard ---
st.title("Mobile Money Fraud Detection Dashboard")

2025-03-13 13:08:21.256 
  command:

    streamlit run C:\Users\G_BOOTS\anaconda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [17]:
# Sidebar for navigation
st.sidebar.header("Navigation")
page = st.sidebar.selectbox("Choose a page", ["Overview", "Predictions", "Feature Importance"])

In [19]:
# Overview Page
if page == "Overview":
    st.header("Model Performance Overview")
    st.write("### Class Distribution")
    st.write(f"Training (Original): {np.bincount(y_train)} (Legit, Fraud)")
    st.write(f"Training (SMOTE): {np.bincount(y_train_smote)} (Legit, Fraud)")
    st.write(f"Test: {np.bincount(y_test)} (Legit, Fraud)")

    st.write("### Classification Report")
    report = classification_report(y_test, y_pred, target_names=['Legit', 'Fraud'], output_dict=True)
    st.write(pd.DataFrame(report).transpose())

    st.write("### Confusion Matrix")
    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots()
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title('Confusion Matrix')
    st.pyplot(fig)

ValueError: Number of classes, 1, does not match size of target_names, 2. Try specifying the labels parameter

In [ ]:
# Predictions Page
elif page == "Predictions":
    st.header("Transaction Predictions")
    st.write("Showing test set predictions (Legit = 0, Fraud = 1)")
    st.dataframe(df_test)

In [ ]:
# Feature Importance Page
elif page == "Feature Importance":
    st.header("Feature Importance")
    feature_importance = pd.DataFrame({
        'Feature': features,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    st.write(feature_importance)

    fig, ax = plt.subplots()
    sns.barplot(x='Importance', y='Feature', data=feature_importance, ax=ax)
    ax.set_title('Feature Importance in Fraud Detection')
    st.pyplot(fig)

In [ ]:
# Footer
st.sidebar.write("Built with Streamlit by [Your Name]")

In [ ]:
# Console output for verification
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Legit', 'Fraud']))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nPredictions saved as 'fraud_predictions.csv'")